In [1]:
DATA_NAME = 'syntheticMRI2D-sagittal' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_vertical_wavelet_sagittal_syntheticMRI2D.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,3,4,5,6,7,8,9
3,1191655.83,-13.64,4.19,-10.12,-0.21,-0.28,-0.01
4,-13.64,1271179.36,-14.90,-9.39,6.45,0.55,0.14
5,4.19,-14.90,363011.78,1.40,2.64,1.46,-0.12
6,-10.12,-9.39,1.40,119355.95,-0.06,-0.04,-0.03
7,-0.21,6.45,2.64,-0.06,25375.06,0.13,0.04
8,-0.28,0.55,1.46,-0.04,0.13,2232.18,0.01
9,-0.01,0.14,-0.12,-0.03,0.04,0.01,107.69


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,3,4,5,6,7,8,9
3,1.00000,-0.00001,0.00001,-0.00003,-0.00000,-0.00001,-0.00000
4,-0.00001,1.00000,-0.00002,-0.00002,0.00004,0.00001,0.00001
5,0.00001,-0.00002,1.00000,0.00001,0.00003,0.00005,-0.00002
6,-0.00003,-0.00002,0.00001,1.00000,-0.00000,-0.00000,-0.00001
7,-0.00000,0.00004,0.00003,-0.00000,1.00000,0.00002,0.00002
8,-0.00001,0.00001,0.00005,-0.00000,0.00002,1.00000,0.00002
9,-0.00000,0.00001,-0.00002,-0.00001,0.00002,0.00002,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

36.59325105161713

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.55593159e+11 1.79641336e+11 1.75808636e+10 1.77512703e+09
 7.12679517e+07 4.15343897e+05 1.33372053e-28]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,-0.562513,0.826661,-0.013887,-0.004209,-0.000882,-0.000078,-0.000004
1,0.823803,0.558902,-0.090773,-0.026559,-0.005570,-0.000490,-0.000024
2,0.065088,0.060449,0.991455,-0.093932,-0.017372,-0.001516,-0.000074
3,0.025284,0.023682,0.089587,0.992466,-0.075802,-0.006268,-0.000303
4,0.007127,0.006676,0.023454,0.073328,0.995973,-0.044832,-0.002137
5,0.000935,0.000875,0.003063,0.009330,0.044040,0.997811,-0.048330
6,0.000090,0.000084,0.000296,0.000901,0.004237,0.048183,0.998829


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[1.5625129213675257,
 0.44109762829754484,
 0.008545493077165789,
 0.007533993580111753,
 0.0040265960382867005,
 0.002189058916408082,
 0.0011709020622931998]